In [5]:
import pandas as pd
import numpy as np
from bff_processor.data_tools import regex_select, get_files, make_df
from bff_processor.bff_meta import preselection, band_cut, isin, identity, all_reg, jet_variations
from bff_processor.bff_cuts import *
from glob import glob
import re

In [21]:
era = 2018
regex_select = ".+/tw_{}_(.+)\.csv".format(era)
files = [x for x in glob('data/*') if re.match(regex_select, x)]
DY = [x for x in files if re.match('.+ZTo(?:Mu|EE).+', x)]
ST = [x for x in files if re.match('.+top.csv', x)]
VB = [x for x in files if re.match('.+mc_(?:ww|wz|zz)', x)]
TT = [x for x in files if re.match('.+ttbar', x)]
BFF = [x for x in files if re.match('.+BFFZp', x)]
data = [x for x in files if re.match('.+_data_', x)]
assert len(files) == len(DY+ST+TT+VB+data+BFF), "duplicate or uncaught file"

In [22]:
sample_types = [
['.+ZTo(?:Mu|EE).+', 'bck'],
['.+top.csv', 'bck'],
['.+mc_(?:ww|wz|zz)', 'bck'],
['.+ttbar', 'bck'],
['.+BFFZp', 'sig'],
['.+_data_', 'data']
]

In [23]:
def process_file(file):
    name = re.findall(regex_select, file)[0]
    #get stuff for bff samples
    dbs = re.findall('dbs(\dp\d+)', name)
    mass = re.findall('_M_(\d+)', name)
    if len(dbs) == 1 : dbs = float(dbs[0].replace('p','.'))
    else: dbs = None
    if len(mass) == 1 : mass = int(mass[0])
    else: mass = None
    type_label = 0
    for re_sample, type_sample in sample_types:
        if re.match(re_sample, file): 
            type_label = type_sample
            break
            
    #open file and filter out events with bff selection
    df = preselection()(pd.read_csv(file))
    
    for reg in df.filter(regex='(?:SR|CR)\d+_.+'):
        nJets, jv = re.findall('(?:SR|CR)(\d)\d*_(.+)', reg)[0]
        df[reg] = df[reg]*bff_no_tmb_value[nJets](df, jv)
    selected_events = df.filter(regex='(?:SR|CR)\d+_.+').sum(axis=1)>0
    df['deltaR'] = df[["minGoodJetElDR","minGoodJetMuDR"]].min(axis=1)
    deltaR = df['deltaR'] > 0.4
    print( "{} remaining".format(   (df.filter(regex='(?:SR|CR)\d+_.+').sum(axis=1)>0).mean()))
    df = df[selected_events & deltaR]
    #remove unnamed column from index, probably a better way, but ok for now
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    #add metadata
    df['name'] = name
    df['dbs'] = dbs
    df['mass'] = mass
    df['type'] = type_label
    return df


In [24]:
combined_df = pd.DataFrame()
for file in files:
    print(file)
    df = process_file(file)
    combined_df = pd.concat([combined_df, df])

data/tw_2018_BFFZprimeToMuMu_M_125_dbs0p04.csv
0.9537698172576546 remaining
data/tw_2018_BFFZprimeToMuMu_M_150_dbs0p04.csv
0.9452616095818449 remaining
data/tw_2018_BFFZprimeToMuMu_M_175_dbs0p04.csv
0.9359595821669967 remaining
data/tw_2018_BFFZprimeToMuMu_M_200_dbs0p04.csv
0.9270585850851809 remaining
data/tw_2018_BFFZprimeToMuMu_M_200_dbs0p5.csv
0.9288025114428814 remaining
data/tw_2018_BFFZprimeToMuMu_M_200_dbs1p0.csv
0.9328932307610402 remaining
data/tw_2018_BFFZprimeToMuMu_M_250_dbs0p04.csv
0.9018579642248722 remaining
data/tw_2018_BFFZprimeToMuMu_M_300_dbs0p04.csv
0.8762545736604169 remaining
data/tw_2018_BFFZprimeToMuMu_M_350_dbs0p04.csv
0.8416799161291666 remaining
data/tw_2018_BFFZprimeToMuMu_M_350_dbs0p5.csv
0.8425273305764346 remaining
data/tw_2018_BFFZprimeToMuMu_M_350_dbs1p0.csv
0.8462707159634052 remaining
data/tw_2018_BFFZprimeToMuMu_M_400_dbs0p04.csv
0.7943406416856383 remaining
data/tw_2018_BFFZprimeToMuMu_M_450_dbs0p04.csv
0.7339226137975994 remaining
data/tw_2018_BFF

In [25]:
# convert to bool
combined_df['TriggerWeight'] = combined_df['TriggerWeight'].astype(bool)

In [26]:
import pyarrow.feather as feather
feather.write_feather(combined_df, 'data/combined_{}.feather'.format(era))

In [27]:
combined_df.to_parquet('data/combined_{}.parquet'.format(era))